# Download libraries

In [ ]:
!pip install pandas requests seaborn IPython flask requests_oauthlib pytz tweepy statsmodels plotly wordcloud textblob nltk requests_oauthlib flask pyspark dask textblob sklearn

# Importing libraries

In [ ]:
# Data Manipulation and Analysis
import pandas as pd
import dask.dataframe as dd
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

# File and Directory Operations
import os
import csv

# Date and Time Handling
import datetime
from datetime import datetime, timedelta
import time
from pytz import timezone

# System-related
from concurrent.futures import ThreadPoolExecutor
import sys
import socket

# Twitter API
import tweepy
import json
import requests_oauthlib

# Plotting and Visualization
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Natural Language Processing
import nltk
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from textblob import TextBlob

# Logging
import logging

# Spark
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SQLContext, SparkSession

# Web Development (Flask)
from flask import Flask, jsonify, request, render_template
from IPython.display import IFrame

# Other Dependencies
import ast
import re
import warnings
import requests

# Loading the credentials

In [ ]:
try:
    # Load the Twitter API credentials from the config file
    with open('config.json', 'r') as f:
        config = json.load(f)
        consumer_key = config['consumer_key']
        consumer_secret = config['consumer_secret']
        access_token = config['access_token']
        access_token_secret = config['access_token_secret']

    # Verify the Twitter API credentials
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    try:
        api = tweepy.API(auth, wait_on_rate_limit=True)
        user = api.verify_credentials()
        logging.info("Twitter API connection successful.")
    except tweepy.error.TweepError as e:
        logging.error("Error: Failed to verify Twitter API credentials.")
        logging.error(str(e))

except FileNotFoundError:
    logging.error("Error: The config file 'config.json' does not exist.")

except json.JSONDecodeError as e:
    logging.error("Error: Failed to load Twitter API credentials from 'config.json'.")
    logging.error(str(e))

except Exception as e:
    logging.error("An error occurred during a Twitter API connection.")
    logging.error(str(e))

# Disabling warnings

In [ ]:
warnings.filterwarnings("ignore")

# Logger

In [ ]:
# Get today's date
today = datetime.now().date()

# Create a log file with today's date in the name
log_file = f"TwitterAPI_{today}.log"

# Configure the root logger
logging.basicConfig(level=logging.INFO)

# Create a FileHandler and set its formatter
file_handler = logging.FileHandler(log_file)
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s','%Y-%m-%d %H:%M:%S'))

# Attach the FileHandler to the root logger
logging.getLogger().addHandler(file_handler)

# Downloading and save twitter data
The free tier of the twitter API holds the limitation of:</br>
<b>**10 Day tweet history limit </br>
**1500 tweet request limit per 900sec circ. </b>

In [ ]:
# Define the topic and initial date range
topic = "(ios OR apple OR AAPL OR iphone OR ipad)"
start_date = today - timedelta(days=91)

# Create a loop to run for 91 days
for _ in range(91):
    # Calculate the end date for the current iteration
    end_date = start_date + timedelta(days=1)
    
    # Format the dates as strings
    start_date_str = start_date.strftime("%Y-%m-%d")
    end_date_str = end_date.strftime("%Y-%m-%d")
    
    # Define the search query with the current date range
    query = f"{topic} until:{end_date_str} since:{start_date_str}"
    
    # Function to fetch tweets for a given query
    def fetch_tweets(query):
        tweets = []
        for tweet in tweepy.Cursor(api.search_tweets, q=query, lang='en', tweet_mode='extended').items(3000):
            tweets.append({
                'Date': tweet.created_at.date(),
                'Tweet': tweet.full_text,
                'User_Location': tweet.user.location  # Added line to fetch user location
            })
        return tweets
    
    try:
        # Fetch tweets on the specified topic using multithreading
        with concurrent.futures.ThreadPoolExecutor() as executor:
            tweets = executor.submit(fetch_tweets, query).result()
        
        if len(tweets) > 0:
            msg = "Tweets downloaded successfully for the date range: {} to {}"
            logging.info(msg.format(start_date_str, end_date_str))
            
            # Convert the tweets list into a DataFrame
            df_new = pd.DataFrame(tweets)
            
            # Check if the CSV file already exists
            if os.path.isfile('tweets.csv'):
                # Read the existing data from the CSV file
                try:
                    df_existing = pd.read_csv('tweets.csv')
                    
                    # Check if the existing DataFrame has any columns
                    if df_existing.columns.empty:
                        # Handle the case when the CSV file is empty
                        df_existing = pd.DataFrame()
                        
                except EmptyDataError:
                    # Handle the case when the CSV file is empty
                    df_existing = pd.DataFrame()
                
                # Check if the existing DataFrame is empty
                if df_existing.empty:
                    # Save the new DataFrame to a new CSV file
                    df_new.to_csv('tweets.csv', index=False)
                    logging.info("New CSV file created with the downloaded tweets.")
                else:
                    # Concatenate the existing and new data
                    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
                    
                    # Save the combined DataFrame to the CSV file
                    df_combined.to_csv('tweets.csv', index=False)
                    logging.info("Tweets appended to the existing CSV file.")
            else:
                # Save the new DataFrame to a new CSV file
                df_new.to_csv('tweets.csv', index=False)
                logging.info("New CSV file created with the downloaded tweets.")
        else:
            msg = "No tweets found for the date range: {} to {}"
            logging.info(msg.format(start_date_str, end_date_str))
            
    except tweepy.TweepyException as e:
        if e.api_code == 88:
            # Rate limit reached, wait for the specified duration
            wait_time = int(e.response.headers['Retry-After'])
            msg = "Rate limit reached. Sleeping for: {} seconds."
            logging.info(msg.format(wait_time))
            time.sleep(wait_time)
        logging.error("Error: Failed to download tweets.")
        logging.error(e)
    
    # Update the start date for the next iteration
    start_date = end_date

# Log data

In [ ]:
# Read the log file into a DataFrame
log_df = pd.read_csv(log_file, sep=":", names=["Timestamp","Type","Message","Sleep"])

# Define the CSV file name for saving the log DataFrame
csv_file = f"TwitterAPI_{today}.csv"

# Check if the CSV file already exists
if os.path.isfile(csv_file):
    # Append the log DataFrame to the existing CSV file
    log_df.to_csv(csv_file, mode='a', header=False, index=False)
else:
    # Save the log DataFrame to a new CSV file
    log_df.to_csv(csv_file, index=False)

# Display the log DataFrame
log_df

# Cleaning tweets

In [ ]:
try:
    # Load the tweets data from the CSV file
    df_tweets = pd.read_csv('tweets.csv')

    # Convert the 'Tweet' column to string type
    df_tweets['Tweet'] = df_tweets['Tweet'].astype(str)

    # Compile the regular expressions
    remove_chars_regex = re.compile(r'[^\w\s]')
    remove_links_regex = re.compile(r'http\S+|www\S+')
    remove_usernames_regex = re.compile(r'@[^\s]+')

    # Function to clean a single tweet
    def clean_tweet(tweet):
        # Remove unnecessary characters and links
        cleaned_tweet = remove_chars_regex.sub('', tweet)
        cleaned_tweet = remove_links_regex.sub('', cleaned_tweet)

        # Remove Twitter usernames
        cleaned_tweet = remove_usernames_regex.sub('', cleaned_tweet)

        # Remove non-English words
        cleaned_words = []
        english_words = set(words.words())
        for word in cleaned_tweet.split():
            if word.lower() in english_words:
                cleaned_words.append(word)
        cleaned_tweet = ' '.join(cleaned_words)

        return cleaned_tweet

    # Function for multithreaded tweet cleaning
    def multithreaded_cleaning(tweet):
        return clean_tweet(tweet)

    # Clean the tweets column using multithreading
    with concurrent.futures.ThreadPoolExecutor() as executor:
        cleaned_tweets = executor.map(multithreaded_cleaning, df_tweets['Tweet'])

    # Update the 'Tweet' column with cleaned tweets
    df_tweets['Tweet'] = list(cleaned_tweets)

    # Remove rows with empty tweet values
    df_tweets = df_tweets.dropna(subset=['Tweet'])

    # Save the cleaned tweets back to the CSV file
    df_tweets.to_csv('tweets.csv', index=False)

    # Log the execution of the tweet cleaning process
    logging.info("Tweet cleaning process completed.")

except Exception as e:
    # Log any exceptions or errors that occur
    logging.error("Error occurred during tweet cleaning process.")
    logging.error(str(e))

# Tweet EDA

In [ ]:
# Read the tweets data from the CSV file
df_tweets = pd.read_csv('tweets.csv')

# Convert the 'Date' column to datetime if it's not already
df_tweets['Date'] = pd.to_datetime(df_tweets['Date'])

# Perform basic exploratory data analysis (EDA)
print("Number of tweets:", len(df_tweets))
print("Columns:", df_tweets.columns)
print("Sample tweets:")
print(df_tweets.head())

# Visualize the downloaded tweets
import matplotlib.pyplot as plt

# Create a grouped subplot for visualizations
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 8))
fig.suptitle('Visualization of Downloaded Tweets')

# Plot 1: Number of tweets per day
tweets_per_day = df_tweets['Date'].value_counts().sort_index()
axes[0].plot(tweets_per_day.index, tweets_per_day.values)
axes[0].set_title('Number of Tweets per Day')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Count')

# Plot 2: Top 10 user locations
top_locations = df_tweets['User_Location'].value_counts().head(10)
axes[1].bar(top_locations.index, top_locations.values)
axes[1].set_title('Top 10 User Locations')
axes[1].set_xlabel('Location')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=45)

# Plot 3: Tweet length distribution
df_tweets.dropna(subset=['Tweet'], inplace=True)  # Drop rows with missing 'Tweet' values
tweet_lengths = df_tweets['Tweet'].apply(len)
axes[1].hist(tweet_lengths, bins=20)
axes[1].set_title('Tweet Length Distribution')
axes[1].set_xlabel('Tweet Length')
axes[1].set_ylabel('Frequency')

# Adjust the spacing between subplots
fig.tight_layout()

# Display the plots
plt.show()

# Sentiment analysis

In [ ]:
try:
    # Load the tweets from the CSV file
    df = pd.read_csv('tweets.csv')

    # Convert 'Tweet' column to string type
    df['Tweet'] = df['Tweet'].astype(str)

    # Perform sentiment analysis using TextBlob
    df['sentiment'] = df['Tweet'].apply(lambda x: TextBlob(x).sentiment.polarity)

    # Classify sentiment as positive, negative, or neutral
    df['sentiment_label'] = df['sentiment'].apply(lambda x: 'Positive' if x > 0 else 'Negative' if x < 0 else 'Neutral')

    # Save the updated DataFrame to CSV
    df.to_csv('tweets_sentiment.csv', index=False)

except pd.errors.EmptyDataError:
    # Handle the case when the CSV file is empty
    logging.error("Error: The CSV file is empty.")

except FileNotFoundError:
    # Handle the case when the CSV file is not found
    logging.error("Error: The CSV file 'tweets.csv' does not exist.")

except Exception as e:
    # Handle any other exceptions that may occur
    logging.error("An error occurred during sentiment analysis and classification.")
    logging.error(str(e))

# Most frequent words wordcloud

In [ ]:
# Generate word cloud of most frequent words
stop_words = set(stopwords.words('english'))

# Convert float values in 'Tweet' column to strings
df_tweets['Tweet'] = df_tweets['Tweet'].astype(str)

# Generate word cloud
wordcloud = WordCloud(stopwords=stop_words, background_color='white').generate(' '.join(df_tweets['Tweet']))

# Plot the word cloud
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Most Frequent Words in Tweets')
plt.axis('off')
plt.show()

# Time series forecast of the sentiment

In [ ]:
# Load the tweet sentiment data from the CSV file
df = pd.read_csv('tweets_sentiment.csv', parse_dates=['Date'])

# Set the 'Date' column as the index
df.set_index('Date', inplace=True)

# Split the data into train and test sets
train_size = int(len(df) * 0.8)
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# Ensure the index of test_data is a datetime index and sorted
test_data.index = pd.to_datetime(test_data.index)
test_data = test_data.sort_index()

# Define the model configurations
arima_order = (1, 0, 1)
sarimax_order = (1, 0, 1)

# Initialize the models
with ThreadPoolExecutor() as executor:
    arima_future = executor.submit(ARIMA, train_data['sentiment'], order=arima_order)
    sarimax_future = executor.submit(SARIMAX, train_data['sentiment'], order=sarimax_order)
    arima_model = arima_future.result().fit()
    sarimax_model = sarimax_future.result().fit()

# Generate predictions for the entire dataset
with ThreadPoolExecutor() as executor:
    arima_predictions_future = executor.submit(arima_model.predict, start=0, end=len(df)-1, dynamic=False)
    sarimax_predictions_future = executor.submit(sarimax_model.predict, start=0, end=len(df)-1)
    arima_predictions = arima_predictions_future.result()
    sarimax_predictions = sarimax_predictions_future.result()

# Evaluate model performance
arima_rmse = mean_squared_error(df['sentiment'], arima_predictions, squared=False)
sarimax_rmse = mean_squared_error(df['sentiment'], sarimax_predictions, squared=False)

# Create Plotly figure for the original plot
fig, ax = plt.subplots(figsize=(10, 6))

# Plot actual sentiment data
ax.plot(df.index, df['sentiment'], label='Actual')

# Plot forecasted sentiment data
ax.plot(df.index, arima_predictions, label=f'ARIMA Forecast (RMSE: {arima_rmse:.2f})')
ax.plot(df.index, sarimax_predictions, label=f'SARIMAX Forecast (RMSE: {sarimax_rmse:.2f})')

# Set plot labels and legend
ax.set_title('Time Series Forecast of Sentiment')
ax.set_xlabel('Date')
ax.set_ylabel('Sentiment')
ax.legend()

# Create subplots for each model's visualization
fig, axs = plt.subplots(3, 1, figsize=(10, 18))

# Plot original data
axs[0].plot(df.index, df['sentiment'], label='Actual')
axs[0].set_title('Original Sentiment Data')
axs[0].set_xlabel('Date')
axs[0].set_ylabel('Sentiment')
axs[0].legend()

# Plot ARIMA forecast
axs[1].plot(df.index, arima_predictions, label=f'ARIMA Forecast (RMSE: {arima_rmse:.2f})')
axs[1].set_title('ARIMA Forecast of Sentiment')
axs[1].set_xlabel('Date')
axs[1].set_ylabel('Sentiment')
axs[1].legend()

# Plot SARIMAX forecast
axs[2].plot(df.index, sarimax_predictions, label=f'SARIMAX Forecast (RMSE: {sarimax_rmse:.2f})')
axs[2].set_title('SARIMAX Forecast of Sentiment')
axs[2].set_xlabel('Date')
axs[2].set_ylabel('Sentiment')
axs[2].legend()

# Adjust spacing between subplots
plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
# Generate 1-week, 1-month, and 3-month forecasts using the SARIMAX model
forecast_1w = sarimax_model.get_forecast(steps=7)
forecast_1m = sarimax_model.get_forecast(steps=30)
forecast_3m = sarimax_model.get_forecast(steps=90)

# Get the forecasted mean values
forecast_1w_mean = forecast_1w.predicted_mean
forecast_1m_mean = forecast_1m.predicted_mean
forecast_3m_mean = forecast_3m.predicted_mean

# Create a DataFrame for the forecast data
forecast_df = pd.DataFrame({
    '1 Week Forecast': forecast_1w_mean,
    '1 Month Forecast': forecast_1m_mean,
    '3 Months Forecast': forecast_3m_mean
})

# Format the DataFrame for better readability
formatted_forecast_df = forecast_df.applymap('{:.2f}'.format)

# Display the formatted forecast DataFrame
print("Forecasted Sentiment:")
print(formatted_forecast_df)

# Sentiment EDA

In [ ]:
# Read the tweets data from the CSV file
df_tweets = pd.read_csv('tweets_sentiment.csv')

# Perform basic exploratory data analysis (EDA)
print("Basic Exploratory Data Analysis:")
print("--------------------------------")
print(f"Number of tweets: {len(df_tweets):,}")
print("\n Columns:")
for column in df_tweets.columns:
    print(f" - {column}")
df_tweets.head(5)

In [ ]:
# Calculate the counts of sentiment labels
sentiment_label_counts = df_tweets['sentiment_label'].value_counts()

# Define the Twitter UI inspired color palette
color_palette = ['#1DA1F2', '#FFAD1F', '#17BF63']  

# Create subplots for multiple visualizations
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot 1: Pie chart for sentiment distribution
axes[0, 0].pie(sentiment_label_counts, labels=sentiment_label_counts.index, autopct='%1.1f%%', colors=color_palette)
axes[0, 0].set_title('Sentiment Distribution by Category')
axes[0, 0].legend(title='Category')
axes[0, 0].axis('equal')

# Plot 2: Bar chart for sentiment counts
axes[0, 1].bar(sentiment_label_counts.index, sentiment_label_counts, color=color_palette)
axes[0, 1].set_title('Sentiment Counts')
axes[0, 1].set_xlabel('Sentiment Category')
axes[0, 1].set_ylabel('Count')

# Plot 3: Histogram of tweet lengths
axes[1, 0].hist(df_tweets['Tweet'].str.len(), bins=20, color='#1DA1F2')
axes[1, 0].set_title('Distribution of Tweet Lengths')
axes[1, 0].set_xlabel('Tweet Length')
axes[1, 0].set_ylabel('Count')

# Plot 4: Line chart of sentiment distribution over time
df_tweets['Date'] = pd.to_datetime(df_tweets['Date'])
df_sentiment_counts = df_tweets.groupby(['Date', 'sentiment_label']).size().unstack()
df_sentiment_counts.plot(ax=axes[1, 1], marker='o')
axes[1, 1].set_title('Sentiment Distribution Over Time')
axes[1, 1].set_xlabel('Date')
axes[1, 1].set_ylabel('Count')
axes[1, 1].legend(title='Sentiment')

# Adjust spacing between subplots
plt.tight_layout()

# Show the plots
plt.show()

# Most frequent words by sentiment

In [ ]:
# Preprocess tweets by filling missing values
df_tweets['Tweet'].fillna('', inplace=True)

# Tokenize the tweets into words
tokenized_words = [word.lower() for tweet in df_tweets['Tweet'] for word in word_tokenize(tweet)]

# Calculate the frequency distribution of words
freq_dist = FreqDist(tokenized_words)
most_common = freq_dist.most_common(20)

# Create a bar chart for the most frequent words by sentiment
fig = go.Figure()
color_palette = ['#1DA1F2', '#FFAD1F', '#17BF63']  # Twitter UI inspired color palette

for i, sentiment in enumerate(df_tweets['sentiment_label'].unique()):
    sentiment_words = [word for word, count in most_common if word.lower() in tokenized_words and df_tweets.loc[df_tweets['sentiment_label'] == sentiment, 'Tweet'].str.contains(word, case=False).any()]
    fig.add_trace(go.Bar(x=sentiment_words, y=[freq_dist[word.lower()] for word in sentiment_words], name=sentiment, marker_color=color_palette[i]))

# Update the layout
fig.update_layout(title='Most Frequent Words by Sentiment', xaxis={'title': 'Word'}, yaxis={'title': 'Count'})

# Show the interactive plot
fig.show()


# Twitter Streaming and Spark Processing

In [ ]:
# Create an OAuth1 authentication object
my_auth = requests_oauthlib.OAuth1(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET)

# Function to get tweets from the Twitter API
def get_tweets():
    # Set the URL and query parameters for filtering tweets
    url = 'https://stream.twitter.com/1.1/statuses/filter.json'
    query_data = [('language', 'en'), ('track', 'ios OR apple OR AAPL OR iphone OR ipad')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data])
    
    # Send the request to the Twitter API with authentication
    response = requests.get(query_url, auth=my_auth, stream=True)
    return response

# Function to send tweets to the TCP connection
def send_tweets_to_spark(http_resp, tcp_connection):
    # Iterate over the response lines from the Twitter API
    for line in http_resp.iter_lines():
        try:
            # Parse each line as JSON
            full_tweet = json.loads(line)
            tweet_text = full_tweet['text']
            
            # Print the tweet text and send it to the TCP connection
            print("Tweet Text: " + tweet_text)
            print("------------------------------------------")
            tcp_connection.send(tweet_text + '\n')
        except:
            e = sys.exc_info()[0]
            print("Error: %s" % e)

# Set up TCP connection parameters
TCP_IP = "localhost"
TCP_PORT = 9009
conn = None

# Create a TCP socket and bind it to the specified IP and port
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)

print("Waiting for TCP connection...")
conn, addr = s.accept()
print("Connected... Starting to get tweets.")

# Get tweets from the Twitter API
resp = get_tweets()

# Send the tweets to the TCP connection
send_tweets_to_spark(resp, conn)

# Spark Streaming and Data Processing

In [ ]:
# Set up Spark configuration
conf = SparkConf().setAppName("TwitterStreamApp")

# Create Spark context with the above configuration
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")

# Create the Streaming Context from the above Spark context with interval size 2 seconds
ssc = StreamingContext(sc, 2)

# Set a checkpoint to allow RDD recovery
ssc.checkpoint("checkpoint_TwitterApp")

# Read data from port 9009
dataStream = ssc.socketTextStream("localhost", 9009)

# Function to aggregate tag counts
def aggregate_tags_count(new_values, total_sum):
    return sum(new_values) + (total_sum or 0)

# Function to get Spark SQL context instance
def get_sql_context_instance(spark_context):
    if 'sqlContextSingletonInstance' not in globals():
        globals()['sqlContextSingletonInstance'] = SparkSession.builder.getOrCreate()
    return globals()['sqlContextSingletonInstance']

# Function to process RDD
def process_rdd(time, rdd):
    print("----------- %s -----------" % str(time))
    try:
        # Get Spark SQL singleton context from the current context
        sql_context = get_sql_context_instance(rdd.context)
        
        # Convert the RDD to DataFrame
        row_rdd = rdd.map(lambda w: Row(hashtag=w[0], hashtag_count=w[1]))
        hashtags_df = sql_context.createDataFrame(row_rdd)
        
        # Create a temporary view for querying
        hashtags_df.createOrReplaceTempView("hashtags")
        
        # Get the top 10 hashtags using SQL
        hashtag_counts_df = sql_context.sql("SELECT hashtag, hashtag_count FROM hashtags ORDER BY hashtag_count DESC LIMIT 10")
        hashtag_counts_df.show()
        
    except Exception as e:
        print("Error: %s" % str(e))

# Function to send DataFrame data to the dashboard
def send_df_to_dashboard(df):
    try:
        # Extract the hashtags from the DataFrame and convert them into an array
        top_tags = [str(t.hashtag) for t in df.select("hashtag").collect()]
        
        # Extract the counts from the DataFrame and convert them into an array
        tags_count = [p.hashtag_count for p in df.select("hashtag_count").collect()]
        
        # Initialize and send the data through REST API
        url = 'http://localhost:5001/updateData'
        request_data = {'label': str(top_tags), 'data': str(tags_count)}
        response = requests.post(url, data=request_data)
        
        if response.status_code == 200:
            print("Data sent to the dashboard successfully.")
        else:
            print("Failed to send data to the dashboard. Status code:", response.status_code)
    
    except Exception as e:
        print("Error occurred while sending data to the dashboard:", str(e))

        
# Create Spark session
spark = SparkSession.builder.appName("TwitterStreamApp").getOrCreate()

# Create StreamingContext from Spark session with interval size 2 seconds
ssc = StreamingContext(spark.sparkContext, 2)

# Set checkpoint directory for RDD recovery
ssc.checkpoint("checkpoint_TwitterApp")

# Read data from port 9009
dataStream = ssc.socketTextStream("localhost", 9009)

# Split each tweet into words
words = dataStream.flatMap(lambda line: line.split(" "))

# Filter the words to get only hashtags, then map each hashtag to be a pair of (hashtag, 1)
hashtags = words.filter(lambda w: '#' in w).map(lambda x: (x, 1))

# Adding the count of each hashtag to its last count
tags_totals = hashtags.updateStateByKey(aggregate_tags_count)

# Do processing for each RDD generated in each interval
tags_totals.foreachRDD(process_rdd)

# Start the streaming computation
ssc.start()

# Wait for the streaming to finish
ssc.awaitTermination()


# Flask Web Application

In [ ]:
# Flask Web Application

# Set up Flask app
app = Flask(__name__)
labels = []
values = []

@app.route("/")
def get_chart_page():
    """
    Route handler for the main chart page.
    Clears the labels and values and renders the chart.html template.
    """
    global labels, values
    labels = []
    values = []
    return render_template('chart.html', values=values, labels=labels)

@app.route('/refreshData')
def refresh_graph_data():
    """
    Route handler to refresh the graph data.
    Returns the current labels and values as a JSON response.
    """
    global labels, values
    print("Labels now: " + str(labels))
    print("Data now: " + str(values))
    return jsonify(sLabel=labels, sData=values)

@app.route('/updateData', methods=['POST'])
def update_data():
    """
    Route handler to update the data.
    Expects 'label' and 'data' fields in the POST request form.
    Updates the labels and values based on the received data.
    """
    global labels, values
    try:
        if not request.form or 'data' not in request.form or 'label' not in request.form:
            raise ValueError("Invalid data received. Missing 'label' or 'data' field.")

        labels = ast.literal_eval(request.form['label'])
        values = ast.literal_eval(request.form['data'])
        print("Labels received: " + str(labels))
        print("Data received: " + str(values))
        return "success", 201
    
    except Exception as e:
        print("Error occurred while updating data:", str(e))
        return "error", 400

if __name__ == "__main__":
    app.run(host='localhost', port=5001)

# Specify the URL of your Flask app
app_url = 'http://localhost:5001/'

# Embed the Flask app in Jupyter Notebook
IFrame(src=app_url, width='100%', height=600)